# Setup

In [ ]:
# Create a new environment with intelpython_full package
conda create -n meowtech intelpython3_full


#Activate the newly created environment
conda activate meowtech


# Install Intel AI Kit for Tensorflow
conda install intel-aikit-tensorflow


# Install/Upgrade additional packages
pip install ipykernel pandas matplotlib glob tqdm 
pip install --upgrade numpy


# Install the Python audio processing library
pip install --user librosa --force-reinstall


# Creating a IPython kernel using the new conda environment
python -m ipykernel install --user --name=meowtech


# Run Jupyter Notebook
jupyter notebook




# Dataset Prep

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Creating DATASET directory
!mkdir DATASET

In [ ]:
# Downloading RAVDESS dataset
#!wget https://zenodo.org/record/1188976/files/Audio_Song_Actors_01-24.zip
#!wget https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip

#Extracting RAVDESS dataset and adding it to DATASET directory
!unzip /content/drive/MyDrive/RAVDESS/Audio_Song_Actors_01-24.zip -d DATASET
!unzip /content/drive/MyDrive/RAVDESS/Audio_Speech_Actors_01-24.zip -d DATASET

Archive:  /content/drive/MyDrive/RAVDESS/Audio_Song_Actors_01-24.zip
   creating: DATASET/Actor_01/
  inflating: DATASET/Actor_01/03-02-01-01-01-01-01.wav  
  inflating: DATASET/Actor_01/03-02-01-01-01-02-01.wav  
  inflating: DATASET/Actor_01/03-02-01-01-02-01-01.wav  
  inflating: DATASET/Actor_01/03-02-01-01-02-02-01.wav  
  inflating: DATASET/Actor_01/03-02-02-01-01-01-01.wav  
  inflating: DATASET/Actor_01/03-02-02-01-01-02-01.wav  
  inflating: DATASET/Actor_01/03-02-02-01-02-01-01.wav  
  inflating: DATASET/Actor_01/03-02-02-01-02-02-01.wav  
  inflating: DATASET/Actor_01/03-02-02-02-01-01-01.wav  
  inflating: DATASET/Actor_01/03-02-02-02-01-02-01.wav  
  inflating: DATASET/Actor_01/03-02-02-02-02-01-01.wav  
  inflating: DATASET/Actor_01/03-02-02-02-02-02-01.wav  
  inflating: DATASET/Actor_01/03-02-03-01-01-01-01.wav  
  inflating: DATASET/Actor_01/03-02-03-01-01-02-01.wav  
  inflating: DATASET/Actor_01/03-02-03-01-02-01-01.wav  
  inflating: DATASET/Actor_01/03-02-03-01-02-

In [ ]:
# Creating TESS_Toronto_emotional_speech_set_data directory
!mkdir TESS_Toronto_emotional_speech_set_data

# Creating the Actor_26 and Actor_28 directories for TESS pipeline
!mkdir DATASET/Actor_26
!mkdir DATASET/Actor_28

# Extracting TESS dataset and adding it to the TESS_Toronto_emotional_speech_set_data directory
!unzip /content/drive/MyDrive/TESS/dataverse_files.zip -d TESS_Toronto_emotional_speech_set_data

Archive:  /content/drive/MyDrive/TESS/dataverse_files.zip
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_back_angry.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_back_disgust.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_back_fear.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_back_happy.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_back_neutral.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_back_ps.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_back_sad.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_bar_angry.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_bar_disgust.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_bar_fear.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_bar_happy.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_bar_neutral.wav  
  inflating: TESS_Toronto_emotional_speech_set_data/OAF_bar_ps.wav  
 

In [ ]:
# TESS Pipeline - to combine RAVDESS and TESS dataset. This can take some time.

import os
import shutil
import random

TRAINING_FILES_PATH = './DATASET/' #Path of DATASET folder containing RAVDESS dataset
TESS_ORIGINAL_FOLDER_PATH = './TESS_Toronto_emotional_speech_set_data/' #Path of TESS dataset

class TESSPipeline:
    @staticmethod
    def create_tess_folders(path):
        label_conversion = {'01': 'neutral',
                            '03': 'happy',
                            '04': 'sad',
                            '05': 'angry',
                            '06': 'fear',
                            '07': 'disgust',
                            '08': 'ps'}

        for subdir, dirs, files in os.walk(path):
            for filename in files:
                if filename.startswith('OAF'):
                    destination_path = TRAINING_FILES_PATH + 'Actor_28/'
                    old_file_path = os.path.join(os.path.abspath(subdir), filename)
                    actor = '28'
                    base, extension = os.path.splitext(filename)

                    for key, value in label_conversion.items():
                        if base.endswith(value):
                            random_list = random.sample(range(10, 99), 6)
                            file_name = '-'.join([str(i) for i in random_list])
                            file_name_with_correct_emotion = file_name[:6] + key + '-' + file_name[9:] + '-26' + extension
                            new_file_path = destination_path + file_name_with_correct_emotion
                            shutil.copy(old_file_path, new_file_path)

                else:
                    destination_path = TRAINING_FILES_PATH + 'Actor_26/'
                    old_file_path = os.path.join(os.path.abspath(subdir), filename)
                    actor = '26'
                    base, extension = os.path.splitext(filename)

                    for key, value in label_conversion.items():
                        if base.endswith(value):
                            random_list = random.sample(range(10, 99), 6)
                            file_name = '-'.join([str(i) for i in random_list])
                            file_name_with_correct_emotion = (file_name[:6] + key + '-' + file_name[9:] + '-25' + extension).strip()
                            new_file_path = destination_path + file_name_with_correct_emotion
                            shutil.copy(old_file_path, new_file_path)
print('TESS Pipeline Started')
TESSPipeline.create_tess_folders(TESS_ORIGINAL_FOLDER_PATH)
print('TESS Pipeline Completed')

TESS Pipeline Started
TESS Pipeline Completed


In [ ]:
# Removing the RAVDESS file structure heirarchy and directories
!find DATASET/ -name \*.wav -exec cp {} DATASET/ \;
! rm -rf DATASET/Actor_*

In [ ]:
# Confirming if the pipeline was successful
import subprocess
count = subprocess.check_output('find DATASET/ -name "*.wav" | wc -l', shell=True)
if count==b'5252\n':
    print("Combining RAVDESS and TESS dataset SUCCESSFUL")
else:
    print("Combining RAVDESS and TESS dataset FAILED")

Combining RAVDESS and TESS dataset SUCCESSFUL


# Feature Extraction

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import librosa
from tqdm import tqdm

In [ ]:
# Function to load dataset and extracting information from the filename

# Loading Data
def labeler(base_dir_path):
    dir_list = os.listdir(base_dir_path)
    dir_list.sort()

    # RAVDESS dataset format
    df = pd.DataFrame(columns=['path', 'source', 'actor', 'gender', 'intensity', 'statement', 'repetition', 'emotion'])
    count = 0

    # Extracting information from filename
    for file in os.listdir(base_dir_path):
        filename = file.split('.')[0].split('-')
        if(len(filename)==7):
            path = base_dir_path + file
            src = int(filename[1])
            actor = int(filename[-1])
            emotion = int(filename[2])
            if int(actor)%2 == 0:
                gender = "female"
            else:
                gender = "male"

            if filename[3] == '01':
                intensity = 0
            else:
                intensity = 1

            if filename[4] == '01':
                statement = 0
            else:
                statement = 1

            if filename[5] == '01':
                repeat = 0
            else:
                repeat = 1

        df.loc[count] = [path, src, actor, gender, intensity, statement, repeat, emotion]
        count += 1

    labels = []
    for i in range(len(df)):
        if df.emotion[i] == 1:
            label = "_neutral"
        elif df.emotion[i] == 2:
            label = "_calm"
        elif df.emotion[i] == 3:
            label = "_happy"
        elif df.emotion[i] == 4:
            label = "_sad"
        elif df.emotion[i] == 5:
            label = "_angry"
        elif df.emotion[i] == 6:
            label = "_fearful"
        elif df.emotion[i] == 7:
            label = "_disgust"
        elif df.emotion[i] == 8:
            label = "_surprised"
        else:
            label = "_none"

        # Add gender to the label 
        labels.append(df.loc[i,'gender'] + label)

    df['label'] = labels
    
    return df

In [ ]:
# Calling the labeler function
df = labeler("./DATASET/")
df

,path,source,actor,gender,intensity,statement,repetition,emotion,label
0,./DATASET/68-36-03-12-20-35-26.wav,36,26,female,1,1,1,3,female_happy
1,./DATASET/03-01-05-02-01-01-09.wav,1,9,male,1,0,0,5,male_angry
2,./DATASET/03-02-03-02-02-02-12.wav,2,12,female,1,1,1,3,female_happy
3,./DATASET/03-01-05-02-01-01-13.wav,1,13,male,1,0,0,5,male_angry
4,./DATASET/03-01-08-02-01-01-02.wav,1,2,female,1,0,0,8,female_surprised
...,...,...,...,...,...,...,...,...,...
5247,./DATASET/21-72-05-63-92-58-25.wav,72,25,male,1,1,1,5,male_angry
5248,./DATASET/03-01-05-02-02-02-19.wav,1,19,male,1,1,1,5,male_angry
5249,./DATASET/70-67-03-30-71-76-26.wav,67,26,female,1,1,1,3,female_happy
5250,./DATASET/03-02-03-01-01-02-22.wav,2,22,female,0,0,1,3,female_happy


In [ ]:
sss = StratifiedShuffleSplit(n_splits=2, random_state=11, test_size=0.2)
for train_index, test_index in sss.split(df, df.label):
    df_train, df_test = df.iloc[train_index,:], df.iloc[test_index,:]

In [ ]:
df_train.index = df_train.path
df_train = df_train.drop("path", axis=1)


df_test.index = df_test.path
df_test = df_test.drop("path", axis=1)

In [ ]:
# Function to extract features such as MFCC, Mels and Chromas from audio

def extract_features(df):
    # features store all features extracted from the audio file
    # labels is the labels for the audio file
    # names is the name of the audio file
    features = pd.DataFrame(columns=['feature'])
    labels = pd.DataFrame(columns=['label'])
    names = pd.DataFrame(columns=['name'])
    
    for i in tqdm(range(df.shape[0])):
        x, sample_rate = librosa.load(df.index[i])
        
        # feature_set stores all features of the audio file
        feature_set = np.array([])
        
        # MFCC feature extraction
        # No. of MFCC Features = 40 (Default = 20)
        mfccs=np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=40).T, axis=0)
        feature_set=np.hstack((feature_set, mfccs))
        
        ## Chroma feature extraction
        # No. of Chroma Features = 12 (Always)
        stft=np.abs(librosa.stft(x))
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        feature_set=np.hstack((feature_set, chroma))
        
        ## Mel feature extraction
        # No. of Mel Features = 128 (Default = 128)
        mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
        feature_set=np.hstack((feature_set, mel))
        
        # Total features = MFCC(40) + Chroma(12) + Mels(128) = 180
        labels.at[i, 'label'] = df.iloc[i, df.columns.get_loc('label')]
        features.loc[i] = [feature_set]
        names.at[i,'name'] = df.index[i].split('/')[-1]
        
        
    final_data = pd.DataFrame(features['feature'].values.tolist())
    final_data = pd.concat([final_data,labels,names], axis=1)
    
    return final_data

In [ ]:
train_data = extract_features(df_train)
test_data = extract_features(df_test)

100%|██████████| 1051/1051 [03:09<00:00,  5.54it/s]


In [ ]:
train_data = train_data.dropna(how='any',axis=0) 
test_data = test_data.dropna(how='any',axis=0) 

In [ ]:
train_data.to_csv("train_features.csv", index=False)
test_data.to_csv("test_features.csv", index=False)

# Dimensional Deduction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
import pickle
import xgboost as xgb
import optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 12.6 MB/s eta 0:00:00


In [ ]:
#Loading Feature DF from saved CSV Files
train_data_load = pd.read_csv('/content/drive/MyDrive/Features/train_features.csv')
test_data_load = pd.read_csv('/content/drive/MyDrive/Features/test_features.csv')

In [ ]:
scalar = StandardScaler()
scalar.fit(train_data_load.iloc[:, :-2])
scaled_data_train = scalar.transform(train_data_load.iloc[:, :-2])
scaled_data_test = scalar.transform(test_data_load.iloc[:, :-2])

In [ ]:
scaled_data_train.shape

(4201, 180)

In [ ]:
pca = PCA(n_components=78)
pca.fit(scaled_data_train)
x_pca = pca.transform(scaled_data_train)
x_test = pca.transform(scaled_data_test)
x_pca.shape

(4201, 78)

In [ ]:
#TODO Implement LDA if accuracy is low

# Implmenting KNN

In [ ]:
xgbmap = {
  'male_angry':"0",
  'female_surprised':"1",
 'male_neutral':"2",
 'male_surprised':"3",
 'male_calm':"4",
 'female_happy':"5",
 'male_disgust':"6",
 'female_fearful':"7",
 'male_sad':"8",
 'female_neutral':"9",
 'female_angry':"10",
 'female_calm':"11",
 'female_sad':"12",
 'male_fearful':"13",
 'male_happy':"14",
 'female_disgust':"15",
}

In [ ]:
X_train = x_pca
Y_train = train_data_load["label"]
Y_test = test_data_load["label"]
Y_train

0         female_neutral
1       female_surprised
2           male_neutral
3             male_happy
4           male_fearful
              ...       
4196        female_angry
4197        female_happy
4198          female_sad
4199           male_calm
4200        male_neutral
Name: label, Length: 4201, dtype: object

In [ ]:
for i in range(len(Y_train)):
  Y_train[i] =  xgbmap[Y_train[i]]
for i in range(len(Y_test)):
  Y_test[i] =  xgbmap[Y_test[i]]

<ipython-input-117-bace9cf79fc6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train[i] =  xgbmap[Y_train[i]]
<ipython-input-117-bace9cf79fc6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test[i] =  xgbmap[Y_test[i]]


In [ ]:
model = KNeighborsClassifier(n_neighbors=16) 
model.fit(X_train, Y_train)
y_pred = model.predict(x_test)

In [ ]:
cm = confusion_matrix(Y_test, y_pred)

In [ ]:
accuracy = accuracy_score(Y_test, y_pred)*100
print('Accuracy of our model is equal ' + str(round(accuracy, 2)) + ' %.')
print(f1_score(Y_test, y_pred, average='weighted'))

Accuracy of our model is equal 76.4 %.
0.7657681040493961


In [ ]:
pickle.dump(model, open("knn.model", 'wb'))

# Implementing XGB

In [ ]:
xgbmap = {
  'male_angry':0,
  'female_surprised':1,
 'male_neutral':2,
 'male_surprised':3,
 'male_calm':4,
 'female_happy':5,
 'male_disgust':6,
 'female_fearful':7,
 'male_sad':8,
 'female_neutral':9,
 'female_angry':10,
 'female_calm':11,
 'female_sad':12,
 'male_fearful':13,
 'male_happy':14,
 'female_disgust':15,
 
}

In [ ]:
for i in range(len(Y_train)):
  Y_train[i] =  xgbmap[Y_train[i]]

<ipython-input-34-bdfb81863b3c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train[i] =  xgbmap[Y_train[i]]


In [ ]:
for i in range(len(Y_test)):
  Y_test[i] =  xgbmap[Y_test[i]]

<ipython-input-43-fedb21684dbe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test[i] =  xgbmap[Y_test[i]]


In [ ]:
model = xgb.XGBClassifier() 
model.fit(X_train, Y_train)
y_pred = model.predict(x_test)

In [ ]:
Y_test = np.array(Y_test).astype(np.int64)

In [ ]:
cm = confusion_matrix(Y_test, y_pred)

In [ ]:
Y_train = np.array(Y_train).astype(np.float64)

In [ ]:
cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy')

array([0.84542212, 0.77857143, 0.82619048, 0.81904762, 0.8047619 ])

In [ ]:
print(f1_score(Y_test, y_pred, average='weighted'))

0.8590037354533725


In [ ]:
pickle.dump(model, open("xgb.model", 'wb'))

# Implementing MLP

In [ ]:
params={'activation': 'tanh', 'solver': 'adam', 'hidden_layer_sizes': 1442, 'alpha': 0.0027099149530944583, 'batch_size': 298, 'learning_rate': 'invscaling'}
model = MLPClassifier(**params,random_state = 22) 
model.fit(X_train, Y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='tanh', alpha=0.0027099149530944583, batch_size=298,
              hidden_layer_sizes=1442, learning_rate='invscaling',
              random_state=22)

In [ ]:
cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy')

/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warn

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
print(f1_score(Y_test, y_pred, average='weighted'))

0.870721363273517


In [ ]:
pickle.dump(model, open("mlp.model", 'wb'))

# Implementing Model Ensemble

In [ ]:
from sklearn.ensemble import VotingClassifier
import pickle

from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10)
# create the sub models
estimators = []
model1 = pickle.load(open("knn.model", 'rb'))
estimators.append(('knn', model1))
model2 = pickle.load(open("xgb.model", 'rb'))
estimators.append(('xgb', model2))
model3 = pickle.load(open("mlp.model", 'rb'))
estimators.append(('mlp', model3))
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, x_test, Y_test, cv=kfold)
print(results.mean())


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:

0.736460017969452


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
ensemble.fit(X_train, Y_train)
y_pred = ensemble.predict(x_test)
print(f1_score(Y_test, y_pred, average='weighted'))

0.8645081752588462


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
pickle.dump(ensemble, open("ensemble.model", 'wb'))